In [1]:
from IPython.display import Image as pdImage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, os, shutil, itertools, fnmatch, re
from scipy.interpolate import griddata, make_interp_spline, BSpline
import scipy.misc
import tensorflow as tf
import cirq
import seaborn as sns
import sys
import pickle as pkl
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
if sys.platform.startswith('win'):
    code_path = "C:\\Users\\Andrew Patterson\\Google Drive\\PhD\\First Year\\Untitled Folder\\cirq_state_discrimination"
else:
    code_path = "/home/zcapga1/Scratch/state_discrimination/code/"
sys.path.append(code_path)
print(tf.__version__)
print(cirq.__version__)

2.0.0-beta1
0.6.0.dev


In [68]:
def create_args(path: str) -> str:
    args = " --create_outputs --restore_loc=\"{}\" ".format(path)
    
    if re.search(r'tf', path):
        args = args + " --use_tf"
            
    with open(os.path.join(path, 'saved_params.json')) as f:
        params = json.load(f)

#     if params['mu_a'] == 0.5:
#         args = args + ' --file_loc=sigma005_mu05_bconst.json'
#     elif params['mu_a'] == 0.25:
#         args = args + ' --file_loc=sigma005_mu025_bconst.json'
#     else:
    args = args + ' --max_epoch=25 --batch_size=100'
    data_args = ['prop_a', 'b_const', 'a_const', 'mu_a', 'mu_b', 'sigma_a', 'sigma_b']
    for a in data_args:
        args = args + ' --{}={}'.format(a, str(params[a]))
            
    dicts = (params['gate_dict'], params['gate_dict_0'], params['gate_dict_1'])
    dict_path = os.path.join(path, 'temp_dicts.pkl')
    with open(dict_path, 'wb') as f:
        pkl.dump(dicts, f, pkl.HIGHEST_PROTOCOL)
    
    args = args + " --dicts=\"{}\"".format(dict_path)

    other_args = ['cost_error', 'cost_incon', 'no_qubits', 'noise_on', 
                  'noise_prob', 'learning_rate', 'beta1', 'beta2']
    for a in other_args:
        args = args + ' --{}={}'.format(a, str(params[a]))
    return args
            
def generate_output_file(directory: str) -> None:
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for restore_path in folder_list:
        full_path = os.path.join(directory, restore_path)
        if os.path.exists(os.path.join(full_path, 'saved_params.json')):
    
            args = create_args(full_path)

            run_file = os.path.join(code_path, 'run_training.py')
            os.system("python \"" + run_file +"\"" + args)
        

noise_dir = r"C:\Users\Andrew Patterson\Desktop"
path = "C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\new_analysis\\tf_noise_array\\2019_08_01_21_50_31"
if re.search("tf", path):
    print('yes')
args = ' --output_loc=\"{}\" '.format(noise_dir) + create_args(r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\new_analysis\tf_noise_array\2019_08_01_21_50_31")
args

yes


' --output_loc="C:\\Users\\Andrew Patterson\\Desktop"  --create_outputs --restore_loc="C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\new_analysis\\tf_noise_array\\2019_08_01_21_50_31"  --use_tf --max_epoch=25 --batch_size=100 --prop_a=0.3333333333333333 --b_const=True --a_const=False --mu_a=0.5 --mu_b=0.75 --sigma_a=0.15 --sigma_b=0.125 --dicts="C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\new_analysis\\tf_noise_array\\2019_08_01_21_50_31\\temp_dicts.pkl" --cost_error=40.0 --cost_incon=40.0 --no_qubits=4 --noise_on=True --noise_prob=0.0005 --learning_rate=0.001 --beta1=0.9 --beta2=0.999'

In [4]:
path = r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data"
folders  = os.listdir(path)
folders.remove('tf_min_error')
folders.remove('tf_min_error_mu025')
folders.remove('BAD_RUNS')
folders = [os.path.join(path, f) for f in folders]

In [3]:
def label_plot(directory: str) -> None:
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for path in folder_list:
        param_path = os.path.join(directory, path, 'saved_params.json')
        if os.path.exists(param_path):
            plot_path = os.path.join(directory, path, 'outputs')
            with open(param_path) as f:
                param_dict = json.load(f)
            
            probs = np.load(os.path.join(plot_path, 'probs.npy'))
            p_suc = np.average([probs[0][0], probs[1][1]])
            msg = 'Success: {:.2f}, '.format(p_suc)

            param_list = ['cost_error', 'cost_incon', 'noise_on', 'noise_prob', 'mu_a', 'sigma_a', 'job_name']
            if not param_dict['b_const']:
                param_list.extend(['mu_b', 'sigma_b'])

            if param_dict['noise_on'] == 'False':
                param_list.remove('noise_prob')
                
            for i, param in enumerate(param_list):
                msg = msg + '{}: {}, '.format(param, param_dict[param])
                if (i + 1) % 3 == 0:
                    msg = msg + "\n"
            msg = msg[:-2]
            

            font_path = os.path.join("C:\\Users\\Andrew Patterson\\Documents\\PhD\\fonts\\fonts\\ofl\\sourcecodepro", 'SourceCodePro-Light.ttf')
            font = ImageFont.truetype(font_path, size=10)
            plot = Image.open(os.path.join(plot_path, 'bar_graph.png'))

            width, height = plot.size
            cropped = plot.crop((100, 35, 570, 435))
            draw = ImageDraw.Draw(cropped)
            (x,y) = (5,5)
            colour = 'rgb(0, 0, 0)'
            draw.text((x,y), msg, fill=colour, font=font)
            save_path = os.path.join(plot_path, 'plot_labeled.png')
            cropped.save(save_path)
            os.system("convert \"{}\" -fuzz 2% -transparent white \"{}\"".format(save_path, save_path))
        
# folders = ['tf_noise_off', 'tf_old_dicts_noise_array', 'tf_old_dicts_noise_array', 'tf_old_dicts_noise_array', 
#            'tf_noise_array', 'tf_noise_array', 'cirq_training_noise_on_array', 'tf_noise_off_long', 
#            'tf_old_dicts_noise_array_long', 'tf_old_dicts_noise_off_long', 'cirq_training_noise_on_array_long', 
#           'tf_noise_array_long', 'tf_min_error', 'tf_min_error_mu025', 'tf_short_dicts_noise_array', 
#            'tf_min_error_mu025_2_60_sigma_001', 'tf_old_2_60_sigma001', 'tf_old_2_60_sigma001_mu025', 
#            'tf_min_error_2_60', 'tf_noise_array_mu025']
# folders = ['tf_noise_array']
# direc = r'C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data'
# # folders = os.listdir(direc)
# for f in folders:
#     path = os.path.join(direc, f)
#     label_plot(path)

In [4]:
def label_plot_val_levels(directory: str) -> None:
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for path in folder_list:
        param_path = os.path.join(directory, path, 'saved_params.json')
        if os.path.exists(param_path):
            
            with open(param_path) as f:
                param_dict = json.load(f)

            param_list = ['cost_error', 'cost_incon', 'noise_on', 'noise_prob', 'mu_a', 'sigma_a', 'job_name']
            if not param_dict['b_const']:
                param_list.extend(['mu_b', 'sigma_b'])
            noise_levels = os.path.join(directory, path, 'outputs_with_noise_levels')
            levels = [f for f in os.listdir(noise_levels) if os.path.isdir(os.path.join(noise_levels, f))]
            for level in levels:
                probs = np.load(os.path.join(noise_levels, level, 'probs.npy'))
                p_suc = np.average([probs[0][0], probs[1][1]])
                msg = 'Success: {:.2f}, '.format(p_suc)
                for i, param in enumerate(param_list):
                    msg = msg + '{}: {}, '.format(param, param_dict[param])
                    if (i + 1) % 3 == 0:
                        msg = msg + "\n"
                msg = msg.replace('noise_prob', 'training_noise_prob')
                msg = msg + 'validation_noise_prob : {} '.format(level.replace('_', '.'))

                font_path = os.path.join("C:\\Users\\Andrew Patterson\\Documents\\PhD\\fonts\\fonts\\ofl\\sourcecodepro", 'SourceCodePro-Light.ttf')
                font = ImageFont.truetype(font_path, size=10)
                plot = Image.open(os.path.join(noise_levels, level, 'bar_graph.png'))

                width, height = plot.size
                cropped = plot.crop((100, 35, 570, 435))
                draw = ImageDraw.Draw(cropped)
                (x,y) = (5,5)
                colour = 'rgb(0, 0, 0)'
                draw.text((x,y), msg, fill=colour, font=font)
                save_path = os.path.join(noise_levels, level, 'plot_labeled.png')
                cropped.save(save_path)
                os.system("convert \"{}\" -fuzz 1% -transparent white \"{}\"".format(save_path, save_path))

# folders = ['tf_old_2_60_sigma001', 'tf_old_2_60_sigma001_mu025', 'tf_min_error_2_60', 'tf_noise_array_mu025']
# direc = r'C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data'
# for f in folders:
#     path = os.path.join(direc, f)
#     label_plot_val_levels(path)

In [5]:
def nwise(iterable, n):
    ts = itertools.tee(iterable, n)
    for c, t in enumerate(ts):
        next(itertools.islice(t, c, c), None)
    return zip(*ts)

def moving_average(iterable, n):
    yield from (sum(x)/n for x in nwise(iterable, n))
    
def plot_loss_fn(folder, n=6, cutoff=100000):
    """
    Input: Takes a list of folders where models are strored and returns a graph of convergences plotted together 
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('Step')
    ax.set_ylabel('Loss')
    ax.set_title('Training loss, Moving average = {}'.format(n))
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    event_file = fnmatch.filter(files, 'events*')
    steps = []
    loss = []
    for e in tf.compat.v1.train.summary_iterator(os.path.join(folder, event_file[0])):
        for v in e.summary.value:
            if v.tag == 'Training loss':
                if e.step <= cutoff:
                    steps.append(e.step)
                    val = np.frombuffer(v.tensor.tensor_content, dtype=np.float32)[0]
                    loss.append(val)

    param_file = os.path.join(folder, 'saved_params.json')
    with open(param_file, 'r') as f:
        params = json.load(f)
    if params['noise_on'] == "False":
        noise_level = 0
    else:
        noise_level = params['noise_prob']
    tot_cost = float(params['cost_error']) + float(params['cost_incon'])
    loss = np.array(loss)
    loss = loss / tot_cost
    np.save((step, loss), os.path.join(folder, 'outputs', 'loss_fn.npy'))
    loss_ma = moving_average(loss, n)
    step_ma = moving_average(steps, n)
    ax.plot(list(step_ma), list(loss_ma), label=noise_level)
    ax.legend()
    plt.savefig(os.path.join(folder, 'outputs', 'loss_fn.png'))
    plt.show()

def save_loss_fns(directory: str):
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for path in folder_list:
        plot_loss_fn(os.path.join(directory, path))
# save_loss_fns(r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_off_g")

In [6]:
def create_data_frame(directory: str):
    df = pd.DataFrame()
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for f in folder_list:
        path = os.path.join(directory, f)
        probs_path = os.path.join(path, 'outputs', 'probs.npy')
        if os.path.isfile(probs_path):
            with open(os.path.join(path, 'saved_params.json')) as js:
                params = json.load(js)
            probs = np.load(probs_path)

            p_error = np.average([probs[0][1], probs[1][0]])
            p_inc = np.average([probs[0][2], probs[1][2]])
            p_suc = np.average([probs[0][0], probs[1][1]])

            loss = p_error + p_inc

            df = df.append({'P_err': p_error, 'P_inc': p_inc, 'P_suc': p_suc, 'cost_err': params['cost_error'],
                            'cost_inc': params['cost_incon'], 'noise_on': params['noise_on'],
                            'noise_prob': params['noise_prob'], 'learning_rate': params['learning_rate'],
                            'loss': loss, 'folder': str(f)},
                           ignore_index=True)


    numeric = ['cost_err', 'cost_inc', 'noise_prob', 'learning_rate']
    df[numeric] = df[numeric].apply(pd.to_numeric, errors='coerce', axis=1)
    if not os.path.exists(os.path.join(directory, 'output')):
        os.mkdir(os.path.join(directory, 'output'))
    df.to_pickle(os.path.join(directory, 'output', 'dataframe.pkl'))
    f, ax = plt.subplots(figsize=(7,7))
    ax.set(xscale='log', title='Loss with log noise')
    df.plot(x='noise_prob', y='loss', ax=ax, style='k--')
    plt.savefig(os.path.join(directory, 'output', 'loss_noise.png'))
    return df

In [9]:
# df = create_data_frame(r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_array_g")
# f, ax = plt.subplots(figsize=(7,7))
# ax.set(xscale='log', title='Loss with log noise')
# df.plot(x='noise_prob', y='loss', ax=ax, style='k--')

In [7]:
def pre_trained_new_noise_level(restore_directory: str, use_tf: bool, noise_levels: list):
    noise_levels_path = os.path.join(restore_directory, 'outputs_with_noise_levels')
    if not os.path.exists(noise_levels_path):
        os.mkdir(noise_levels_path)
        
    for noise in noise_levels:
        args_str = create_args(restore_directory)
        args_str = re.sub('(--noise_prob=\d*.\d*)', "--noise_prob={}".format(noise), args_str)
        args_str = re.sub('(--noise_on=\w*|\"\w*\")', "--noise_on=True", args_str)
        
        noise_dir = os.path.join(noise_levels_path, re.sub('\.', '_', str(noise)))
        args_str = args_str + ' --output_loc=\"{}\"'.format(noise_dir)
        run_file = os.path.join(code_path, 'run_training.py')
        os.system("python \"" + run_file +"\"" + args_str)
        
def pre_trained_multiple(directory: str, noise_levels: list):
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for path in folder_list:
        # if (not os.path.exists(os.path.join(directory, path, 'outputs_with_noise_levels'))) and os.path.exists(os.path.join(directory, path, 'saved_params.json')):
        if os.path.exists(os.path.join(directory, path, 'saved_params.json')):
            if re.match(r'tf_', path):
                use_tf = True
            else:
                use_tf = False
            pre_trained_new_noise_level(os.path.join(directory, path), use_tf, noise_levels)

noise_levels = [0.0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25]        
# pre_trained_multiple(r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_off_g", True, noise_levels)
# pre_trained_multiple(r"C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_array", noise_levels)

In [8]:
deep=["#4C72B0", "#55A868", "#C44E52",
      "#8172B2", "#CCB974", "#64B5CD"]
muted=["#4878CF", "#6ACC65", "#D65F5F",
       "#B47CC7", "#C4AD66", "#77BEDB"]
pastel=["#92C6FF", "#97F0AA", "#FF9F9A",
        "#D0BBFF", "#FFFEA3", "#B0E0E6"]
bright=["#003FFF", "#03ED3A", "#E8000B",
        "#8A2BE2", "#FFC400", "#00D7FF"]
dark=["#001C7F", "#017517", "#8C0900",
      "#7600A1", "#B8860B", "#006374"]
colorblind=["#0072B2", "#009E73", "#D55E00",
            "#CC79A7", "#F0E442", "#56B4E9"]
pal = dark
noise_pal = {0.0: pal[0], 0.0005: pal[1], 0.001: pal[2], 0.005: pal[3],  0.01: pal[4], 0.05: pal[5], 0.1: pal[0], 0.25:pal[1]}

def create_noise_levels_df(directory: str) -> pd.DataFrame:
    df = pd.DataFrame()
    params_path = os.path.join(directory, 'saved_params.json')
    with open(params_path) as js:
        params = json.load(js)
    noise_folder = os.path.join(directory, 'outputs_with_noise_levels')
    noise_paths = [f for f in os.listdir(noise_folder) if os.path.isdir(os.path.join(noise_folder, f))]
    for p in noise_paths:
        path = os.path.join(noise_folder, p)
        probs = np.load(os.path.join(path, 'probs.npy'))
        
        p_error = np.average([probs[0][1], probs[1][0]])
        p_inc = np.average([probs[0][2], probs[1][2]])
        p_suc = np.average([probs[0][0], probs[1][1]])

        loss = p_error + p_inc
        
        if params['noise_on'] == 'True':
            training_noise = params['noise_prob']
        else:
            training_noise = 0

        df = df.append({'P_err': p_error, 'P_inc': p_inc, 'cost_err': params['cost_error'],
                        'cost_inc': params['cost_incon'],'training_noise': training_noise, 
                        'noise_prob': re.sub('\_', '.', p), 'loss': loss},
                       ignore_index=True)
    return df
        
def log_on_one_plot(directory: str):
    df = pd.DataFrame()
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for f in folder_list:
        if os.path.exists(os.path.join(directory, f, 'saved_params.json')):
            df = df.append(create_noise_levels_df(os.path.join(directory, f)))
    f, ax = plt.subplots(figsize=(7,7))
    #ax.set(xscale='log', title='Loss with log noise')
    
    numeric = ['cost_err', 'cost_inc', 'noise_prob', 'loss']
    df[numeric] = df[numeric].apply(pd.to_numeric, errors='coerce', axis=1)
    for x in [0.0005, 0.005, 0.25]:
        df = df[df.training_noise != x]
    df['training_noise'] = df['training_noise'].astype('category')
    sns.scatterplot(x='noise_prob', y='loss', style='training_noise', data=df, hue='training_noise', palette=noise_pal)
#     for key, group in df.groupby('training_noise'):
#         sns.scatterplot(group['noise_prob'], group['loss'], legend='full')
    if not os.path.exists(os.path.join(directory, 'output')):
        os.mkdir(os.path.join(directory, 'output'))
    #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    ax.set(xscale='log', yscale='log', title='Log loss vs. log validation noise level\nfor different levels of training noise.')
    ax.set(xlim=(0.00045, 0.3), ylim=(0.002,1))
    plt.savefig(os.path.join(directory, 'output', 'log_noise_plot.png'))
    df.to_pickle(os.path.join(directory, 'output', 'noise_df.pkl'))

# log_on_one_plot(r'C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_off_g')    
# log_on_one_plot(r'C:\Users\Andrew Patterson\Documents\PhD\cirq_state_discrimination\checkpoints\myriad_data\tf_noise_array')

In [16]:
if sys.platform.startswith('win'):
    checkpoints = "C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\"
else:
    checkpoints = "/home/zcapga1/Scratch/state_discrimination/training_out/"

# runs = ['tf_noise_off', 'tf_old_dicts_noise_off', 'cirq_training_noise_off', 'tf_noise_array', 'tf_old_dicts_noise_array', 'tf_old_dicts_noise_off']
runs = os.listdir(checkpoints)
rm = ['BAD_RUNS', 'cirq_training_noise_off', 'cirq_training_noise_on_array', 'cirq_training_noise_off_long', 'cirq_training_noise_on_array_long']
for r in rm:
    runs.remove(r)
folder_list = [os.path.join(checkpoints, f) for f in runs]

noise_levels = [0.0, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25]
def run_on_all_folders(folders: list, noise_levels: list):
    for folder in folders:
        generate_output_file(folder)
        label_plot(folder)
        save_loss_fns(folder)
        create_data_frame(folder)
        pre_trained_multiple(folder, noise_levels)
        label_plot_val_levels(folder)
        log_on_one_plot(folder)
        
# run_on_all_folders(folder_list, noise_levels)

In [52]:
def check_probs(directory: str):
    flagged_folders = []
    okay = []
    folder_list = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
    for folder in folder_list:
        full_path = os.path.join(directory, folder)
        if os.path.exists(os.path.join(full_path, 'saved_params.json')):
            with open(os.path.join(full_path, 'saved_params.json'), 'r') as f:
                params = json.load(f)
            tr_noise = params['noise_prob']
            probs = np.load(os.path.join(full_path, 'outputs', 'probs.npy'))
            if (not np.isclose(np.sum(probs[0]), 1, atol=0.0001, rtol=0.1)) or (not np.isclose(np.sum(probs[1]), 1, atol=0.0001, rtol=0.1)):
                flagged_folders.append(directory[90:] + ' tr:{}'.format(tr_noise))
                
            noise_outputs = os.path.join(full_path, 'outputs_with_noise_levels')
            noise_levels = [f for f in os.listdir(noise_outputs) if os.path.isdir(os.path.join(noise_outputs, f))]
            for noise in noise_levels:
                noise_path = os.path.join(noise_outputs, noise)
                probs = np.load(os.path.join(noise_path, 'probs.npy'))
                if (not np.isclose(np.sum(probs[0]), 1, atol=0.0001, rtol=0.1)) or (not np.isclose(np.sum(probs[1]), 1, atol=0.0001, rtol=0.1)):
                    flagged_folders.append(directory[90:] + ' tr:{}, vl:{}'.format(tr_noise, noise.replace('_', '.')))
                    
                    
                    
    return flagged_folders, okay

flagged_out = []
okay_out = []
direc = "C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\new_analysis"
folder_list = [os.path.join(direc, f) for f in os.listdir(direc) if os.path.isdir(os.path.join(direc, f))]

for f in folder_list:
    flagged, okay = check_probs(f)
    flagged_out.append(flagged)
    okay_out.append(okay)
print(flagged_out)

[['new_analysis\\tf_noise_array tr:0.0005, vl:0', 'new_analysis\\tf_noise_array tr:0.005, vl:0', 'new_analysis\\tf_noise_array tr:0.005, vl:0.05', 'new_analysis\\tf_noise_array tr:0.005, vl:0.1', 'new_analysis\\tf_noise_array tr:0.001, vl:0.05', 'new_analysis\\tf_noise_array tr:0.001, vl:0.1', 'new_analysis\\tf_noise_array tr:0.01, vl:0', 'new_analysis\\tf_noise_array tr:0.05, vl:0', 'new_analysis\\tf_noise_array tr:0.1, vl:0', 'new_analysis\\tf_noise_array tr:0.1, vl:0.01', 'new_analysis\\tf_noise_array tr:0.1, vl:0.05', 'new_analysis\\tf_noise_array tr:0.25, vl:0.001', 'new_analysis\\tf_noise_array tr:0.25, vl:0.01', 'new_analysis\\tf_noise_array tr:0.25, vl:0.05', 'new_analysis\\tf_noise_array tr:0.25, vl:0.1'], []]


In [24]:
len("C:\\Users\\Andrew Patterson\\Documents\\PhD\\cirq_state_discrimination\\checkpoints\\myriad_data\\")

90